In [ ]:
pip install tensorflow==2.4.1

In [1]:
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from matplotlib import pyplot as plt

In [2]:
DATASET_PATH = 'C:/Users/user/Desktop/imp/stag2/Stanford_Online_Products/'
TRAIN_FILE = 'Ebay_train.txt'
TEST_FILE = 'Ebay_test.txt'

In [4]:
df_train = pd.read_csv(f'{DATASET_PATH}{TEST_FILE}', sep=' ')
df_train.head()
DATASET_PATH = 'C:/Users/user/Desktop/imp/stag2/Stanford_Online_Products/'
TRAIN_FILE = 'Ebay_train.txt'
TEST_FILE = 'Ebay_test.txt'

In [7]:
from random import sample
from tqdm import tqdm
from random import choice

class DatasetHandler:

    def __init__(self, train_path: str, test_path: str,
                 train_part: float = 1., test_part: float = 1.,
                 batch_size=64,
                 target_shape=(400, 400)):
        self.__target_shape = target_shape

        full_train_data = pd.read_csv(train_path, sep=' ')
        full_test_data = pd.read_csv(test_path, sep=' ')

        part_train_data_indexes = list(full_train_data.index)
        part_test_data_indexes = list(full_test_data.index)

        len_train_part = int(train_part * len(part_train_data_indexes))
        len_test_part = int(test_part * len(part_test_data_indexes))

        train_source = full_train_data.loc[sample(part_train_data_indexes, len_train_part)]
        test_source = full_test_data.loc[sample(part_test_data_indexes, len_test_part)]

        del full_train_data, full_test_data

        # Train/test triplets

        tqdm.write(f'Train generating')
        train_triplets = self.__generate_triplets(train_source)
        self.__train_dataset = self.__seal_dataset(train_triplets)
        self.__train_dataset = self.__train_dataset.batch(batch_size).prefetch(2)
        tqdm.write(f'Test generating')
        test_triplets = self.__generate_triplets(test_source)
        self.__test_dataset = self.__seal_dataset(test_triplets)
        self.__test_dataset = self.__test_dataset.batch(batch_size).prefetch(2)


    def __form_triplet(self, ind: int, data: pd.DataFrame):
        anchor = data.iloc[ind]
        similar_indexes = data.loc[(data.class_id == anchor.class_id) & (data.image_id != anchor.image_id)].index
        if len(similar_indexes) == 0:
            similar_indexes = data.loc[(data.super_class_id == anchor.super_class_id)].index
        positive = data.loc[choice(similar_indexes)]
        different_indexes = data.drop(index=data.loc[data.class_id == anchor.class_id].index).index
        negative = data.loc[choice(different_indexes)]

        return anchor, positive, negative


    def __generate_triplets(self, data: pd.DataFrame):
        triplets = {'anchors': [], 'positive': [], 'negative': []}
        for i in tqdm(range(data.shape[0])):
            anchor, positive, negative = self.__form_triplet(i, data)
            triplets['anchors'].append(f'{DATASET_PATH}{anchor["path"]}')
            triplets['positive'].append(f'{DATASET_PATH}{positive["path"]}')
            triplets['negative'].append(f'{DATASET_PATH}{negative["path"]}')
        return triplets


    def __seal_dataset(self, data: dict):
        anchor_dataset = tf.data.Dataset.from_tensor_slices(data['anchors'])
        positive_dataset = tf.data.Dataset.from_tensor_slices(data['positive'])
        negative_dataset = tf.data.Dataset.from_tensor_slices(data['negative'])

        triplets_path_dataset = tf.data.Dataset.zip((anchor_dataset, positive_dataset, negative_dataset))
        triplets_images_dataset = triplets_path_dataset.map(self.__preprocess_triplets)

        return triplets_images_dataset


    def __preprocess_image(self, filename: tf.Tensor):
        """
        Load the specified file as a JPEG image, preprocess it and
        resize it to the target shape.
            """


        image_string = tf.io.read_file(filename)
        image = tf.image.decode_jpeg(image_string, channels=3)
        image = tf.image.convert_image_dtype(image, tf.float32)
        image = tf.image.resize(image, (400, 400))

        return image


    @tf.autograph.experimental.do_not_convert
    def __preprocess_triplets(self, anchor, positive, negative):
        """
        Given the filenames corresponding to the three images, load and
        preprocess them.
        """

        return (
            self.__preprocess_image(anchor),
            self.__preprocess_image(positive),
            self.__preprocess_image(negative),
        )


    def get_target_shape(self):
        return self.__target_shape

    def get_training_data(self):
        return self.__train_dataset

    def get_validation_data(self):
        return self.__test_dataset

In [8]:
data_handler = DatasetHandler(train_path=f'{DATASET_PATH}{TRAIN_FILE}', train_part=0.2,
                              test_path=f'{DATASET_PATH}{TEST_FILE}', test_part=0.05)

  0%|▏                                                                             | 28/11910 [00:00<01:26, 137.82it/s]

Train generating


  1%|▍                                                                              | 17/3025 [00:00<00:18, 165.04it/s]

Test generating


100%|█████████████████████████████████████████████████████████████████████████████| 3025/3025 [00:19<00:00, 156.88it/s]


In [9]:
train = data_handler.get_training_data()
test = data_handler.get_validation_data()

In [10]:
train

<PrefetchDataset shapes: ((None, 400, 400, 3), (None, 400, 400, 3), (None, 400, 400, 3)), types: (tf.float32, tf.float32, tf.float32)>

In [11]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras import Model
from tensorflow.keras.applications import resnet


class DistanceLayer(layers.Layer):
    """
    This layer is responsible for computing the distance between the anchor
    embedding and the positive embedding, and the anchor embedding and the
    negative embedding.
    """

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, anchor, positive, negative):
        ap_distance = tf.reduce_sum(tf.square(anchor - positive), -1)
        an_distance = tf.reduce_sum(tf.square(anchor - negative), -1)
        return ap_distance, an_distance


class SiameseModel(Model):
    """The Siamese Network model with a custom training and testing loops.
    Computes the triplet loss using the three embeddings produced by the
    Siamese Network.
    The triplet loss is defined as:
       L(A, P, N) = max(‖f(A) - f(P)‖² - ‖f(A) - f(N)‖² + margin, 0)
    """

    def __init__(self, siamese_network, margin=0.5):
        super(SiameseModel, self).__init__()
        self.siamese_network = siamese_network
        self.margin = margin
        self.loss_tracker = metrics.Mean(name="loss")

    def call(self, inputs, **kwargs):
        return self.siamese_network(inputs)

    def train_step(self, data):
        # GradientTape is a context manager that records every operation that
        # you do inside. We are using it here to compute the loss so we can get
        # the gradients and apply them using the optimizer specified in
        # `compile()`.
        with tf.GradientTape() as tape:
            loss = self._compute_loss(data)

        # Storing the gradients of the loss function with respect to the
        # weights/parameters.
        gradients = tape.gradient(loss, self.siamese_network.trainable_weights)

        # Applying the gradients on the model using the specified optimizer
        self.optimizer.apply_gradients(
            zip(gradients, self.siamese_network.trainable_weights)
        )

        # Let's update and return the training loss metric.
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def test_step(self, data):
        loss = self._compute_loss(data)

        # Let's update and return the loss metric.
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def _compute_loss(self, data):
        # The output of the network is a tuple containing the distances
        # between the anchor and the positive example, and the anchor and
        # the negative example.
        ap_distance, an_distance = self.siamese_network(data)

        # Computing the Triplet Loss by subtracting both distances and
        # making sure we don't get a negative value.
        loss = ap_distance - an_distance
        loss = tf.maximum(loss + self.margin, 0.0)
        return loss

    @property
    def metrics(self):
        # We need to list our metrics here so the `reset_states()` can be
        # called automatically.
        return [self.loss_tracker]


class Modell:
    def __init__(self, target_shape):
        base_cnn = resnet.ResNet50(
            weights="imagenet", input_shape=target_shape + (3,), include_top=False
        )

        flatten = layers.Flatten()(base_cnn.output)
        dense1 = layers.Dense(512, activation="relu")(flatten)
        dense1 = layers.BatchNormalization()(dense1)
        dense2 = layers.Dense(256, activation="relu")(dense1)
        dense2 = layers.BatchNormalization()(dense2)
        output = layers.Dense(256)(dense2)

        embedding = Model(base_cnn.input, output, name="Embedding")

        trainable = False
        for layer in base_cnn.layers:
            if layer.name == "conv5_block1_out":
                trainable = True
            layer.trainable = trainable

        anchor_input = layers.Input(name="anchor", shape=target_shape + (3,))
        positive_input = layers.Input(name="positive", shape=target_shape + (3,))
        negative_input = layers.Input(name="negative", shape=target_shape + (3,))

        self.distances = DistanceLayer()(
            embedding(resnet.preprocess_input(anchor_input)),
            embedding(resnet.preprocess_input(positive_input)),
            embedding(resnet.preprocess_input(negative_input)),
        )

        self.siamese_network = Model(inputs=[anchor_input, positive_input, negative_input], outputs=distances)

    def get_siamese_network(self):
        return self.siamese_network

    @staticmethod
    def train(siamese_network, train_data, val_data):
        siamese_model = SiameseModel(siamese_network)
        siamese_model.compile(optimizer=optimizers.Adam(0.0001))
        siamese_model.fit(train_data, epochs=2, validation_data=val_data)

    def test(self):
        """todo"""

    def inference(self):
        """todo"""

In [ ]:
Modell.train(siamese_network, train, test)

In [12]:
test

<PrefetchDataset shapes: ((None, 400, 400, 3), (None, 400, 400, 3), (None, 400, 400, 3)), types: (tf.float32, tf.float32, tf.float32)>

In [28]:
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras import Model
from tensorflow.keras.applications import resnet
anchor_input = layers.Input(name="anchor", shape=data_handler.get_target_shape() + (3,))
positive_input = layers.Input(name="positive", shape=data_handler.get_target_shape() + (3,))
negative_input = layers.Input(name="negative", shape=data_handler.get_target_shape() + (3,))

base_cnn = resnet.ResNet50(
    weights="imagenet", input_shape=data_handler.get_target_shape() + (3,), include_top=False
)

flatten = layers.Flatten()(base_cnn.output)
dense1 = layers.Dense(512, activation="relu")(flatten)
dense1 = layers.BatchNormalization()(dense1)
dense2 = layers.Dense(256, activation="relu")(dense1)
dense2 = layers.BatchNormalization()(dense2)
output = layers.Dense(256)(dense2)

embedding = Model(base_cnn.input, output, name="Embedding")

trainable = False
for layer in base_cnn.layers:
    if layer.name == "conv5_block1_out":
        trainable = True
    layer.trainable = trainable

distances = DistanceLayer()(
    embedding(resnet.preprocess_input(anchor_input)),
    embedding(resnet.preprocess_input(positive_input)),
    embedding(resnet.preprocess_input(negative_input)),
)

siamese_network = Model(
    inputs=[anchor_input, positive_input, negative_input], outputs=distances
)

In [23]:
distances

(<KerasTensor: shape=(None,) dtype=float32 (created by layer 'distance_layer_4')>,
 <KerasTensor: shape=(None,) dtype=float32 (created by layer 'distance_layer_4')>)

In [33]:
siamese_network.predict(anchor_input, positive_input, negative_input)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'

In [24]:
Modell((400, 400)).predict(test)

ValueError: Graph disconnected: cannot obtain value for tensor KerasTensor(type_spec=TensorSpec(shape=(None, 400, 400, 3), dtype=tf.float32, name='negative'), name='negative', description="created by layer 'negative'") at layer "tf.__operators__.getitem_11". The following previous layers were accessed without issue: []

In [ ]:
test

In [ ]:
import pandas as pd
import tensorflow as tf
from random import choice, sample, randint
from tqdm import tqdm

FULL_DATASET_FILE = 'Ebay_info.txt'


class DatasetHandler:
    """DataHandler - класс для получения и обработки StandfordDataset для модели основанной на TripletLoss
    Attributes:
    -----------
    dataset_dir : str
            путь к папке Standford_Online_Products (Пример: ../Data/Standford_Online_Products)
    split_dataset : tuple(int, int)
            отношение частей train и test
    batch_size : int
            количество триплетов в батче
    target_shape : tuple(int, int)
            размер в который будут переведены изображения
    """

    def __init__(self, dataset_dir,
                 split_dataset=(0.8, 0.2),
                 dataset_part=1,
                 batch_size=64,
                 target_shape=(400, 400)):

        self.__target_shape = target_shape
        self.dataset_dir = dataset_dir

        full_dataset = pd.read_csv(f'{self.dataset_dir}/{FULL_DATASET_FILE}', sep=' ')
        self.__dataset_partitions = split_dataset

        df_train, df_test = self.__split_dataset(full_dataset, dataset_part)

        # Train/test triplets

        tqdm.write(f'Train generating')
        train_triplets = self.__generate_triplets(df_train)
        self.__train_dataset = self.__seal_dataset(train_triplets)
        self.__train_dataset = self.__train_dataset.batch(batch_size).prefetch(2)
        tqdm.write(f'Test generating')
        test_triplets = self.__generate_triplets(df_test)
        self.__test_dataset = self.__seal_dataset(test_triplets)
        self.__test_dataset = self.__test_dataset.batch(batch_size).prefetch(2)

    def __split_dataset(self, data: pd.DataFrame, dataset_part: float):
        """
        Деление всего датасета на train/test в зависимости с dataset_part и split_dataset, переданным в параметры класса
        """
        super_classes = list(data.super_class_id.unique())
        df_train = data.copy()
        df_test = data.copy()
        for super_class in super_classes:
            super_class_indexes = list(data.loc[data.super_class_id == super_class].index)
            dropped_index = sample(super_class_indexes, int(dataset_part * len(super_class_indexes)))
            dropped_index = list(set(super_class_indexes) - set(dropped_index))
            train_super_class_indexes = sample(super_class_indexes,
                                               int(self.__dataset_partitions[0] * len(super_class_indexes)))

            test_super_class_indexes = list(set(super_class_indexes) - set(train_super_class_indexes))

            df_train.drop(index=test_super_class_indexes + dropped_index, inplace=True)
            df_test.drop(index=train_super_class_indexes + dropped_index, inplace=True)
        return df_train, df_test

    def __form_triplet(self, ind: int, data: pd.DataFrame):
        """
        Формирование триплета. Для выбранного изображения берется изображение из его класса, если такое отсутствует, то
        берется из суперласса. Отличное от выбранного изображение берется таким, чтобы оно не было в том же классе, что
        и выбранный.
        """
        anchor = data.iloc[ind]
        similar_indexes = data.loc[(data.class_id == anchor.class_id) & (data.image_id != anchor.image_id)].index
        if len(similar_indexes) == 0:
            similar_indexes = data.loc[(data.super_class_id == anchor.super_class_id)].index
        positive = data.loc[choice(similar_indexes)]
        different_indexes = data.drop(index=data.loc[data.class_id == anchor.class_id].index).index
        negative = data.loc[choice(different_indexes)]

        return anchor, positive, negative

    def __generate_triplets(self, data: pd.DataFrame):
        """
        Генерация триплетов, на данном этапе хранятся лишь пути к изображениям
        """
        triplets = {'anchors': [], 'positive': [], 'negative': []}
        for i in tqdm(range(data.shape[0])):
            anchor, positive, negative = self.__form_triplet(i, data)
            triplets['anchors'].append(f'{self.dataset_dir}{anchor["path"]}')
            triplets['positive'].append(f'{self.dataset_dir}{positive["path"]}')
            triplets['negative'].append(f'{self.dataset_dir}{negative["path"]}')
        return triplets

    def __seal_dataset(self, data: dict):
        """
        Получение триплета из целевого изображения, похожего на него и отличного от него.
        """
        anchor_dataset = tf.data.Dataset.from_tensor_slices(data['anchors'])
        positive_dataset = tf.data.Dataset.from_tensor_slices(data['positive'])
        negative_dataset = tf.data.Dataset.from_tensor_slices(data['negative'])

        triplets_path_dataset = tf.data.Dataset.zip((anchor_dataset, positive_dataset, negative_dataset))
        triplets_images_dataset = triplets_path_dataset.map(self.__preprocess_triplets).map(
            self.__augmentation_triplets)

        return triplets_images_dataset

    def __augmentation_triplets(self, anchor, positive, negative):

        """
        Аугментация каждого изображения из триплета
        """

        return (
            self.__augmentation_image(anchor),
            self.__augmentation_image(positive),
            self.__augmentation_image(negative),
        )

    def __augmentation_image(self, image):

        """
        Аугментация изображения
        random_flip_left_right - случайное отражение по оси Y
        random_flip_up_down - случайное отражение по оси X
        random_brightness - случайное изменение яяркости
        random_contrast - случайное изменение контраста
        random_saturation -  случайное изменение насыщенности
        rot90 - переворот на 90 градусов случайное кол-во раз
        """

        aug_image = tf.image.random_flip_left_right(image)
        aug_image = tf.image.random_flip_up_down(aug_image)
        aug_image = tf.image.random_brightness(aug_image, max_delta=0.3)
        aug_image = tf.image.random_contrast(aug_image, lower=0.6, upper=1)
        aug_image = tf.image.random_saturation(aug_image, 0.6, 1)
        aug_image = tf.image.rot90(aug_image, k=randint(0, 3))
        return aug_image

    def __preprocess_image(self, filename: tf.Tensor):
        """
        Загрузка изображения, декодирование, перевод значений в числа с плавающей точкой, а также изменение размера
        """

        image_string = tf.io.read_file(filename)
        image = tf.image.decode_jpeg(image_string, channels=3)
        image = tf.image.convert_image_dtype(image, tf.float32)
        image = tf.image.resize(image, self.__target_shape)

        return image

    @tf.autograph.experimental.do_not_convert
    def __preprocess_triplets(self, anchor, positive, negative):
        """
        Метод для обработки каждого изображения из триплета
        """

        return (
            self.__preprocess_image(anchor),
            self.__preprocess_image(positive),
            self.__preprocess_image(negative),
        )

    """
    Метод для поучения установленного размера изображений
    """

    def get_target_shape(self):
        return self.__target_shape

    """
    Метод ждя получения train dataset
    """

    def get_training_data(self):
        return self.__train_dataset

    """
    Метод для получения test dataset
    
    
    """

    def get_validation_data(self):
        return self.__test_dataset